### V1.5 Unified Auto Model Scoring Code

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
print(pd.__version__)
import re
import datetime
import dateutil.relativedelta
from datetime import datetime
import math
pd.set_option('display.max_columns', None)

import h2o
import os
from h2o.estimators import H2OXGBoostEstimator
from h2o.estimators import H2OGradientBoostingEstimator

pd.options.display.max_columns= 1000
pd.options.display.max_rows= 1000

h2o.init()

1.5.3
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,5 mins 05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,"21 days, 15 hours and 15 minutes"
H2O_cluster_name:,H2O_from_python_unknownUser_nm5nub
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.486 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [2]:
# Get bivar tables
def get_bivar_tables_test(filename, predicted, feature_meta, feature_boundaries, num_features, embed_features, dep_var='pred',
                     weight_column=None):
    bivar_tables = {}
#     print("FILENAME")
#     print(filename)
#     features = pd.read_csv(filename)
    features= filename.copy()

    if predicted is not None:
        features['pred'] = predicted['pred']

    representative_values = {}
    binned_df = features
    for feature in num_features:
        print(feature)
        df = pd.DataFrame()
        df['feature'] = features[feature]
        if weight_column is not None:
            #df['labels'] = features[dep_var] * features[weight_column]
            df['wt'] = features[weight_column]
        else:
            #df['labels'] = features[dep_var]
            df['wt'] = 1

        f = df.loc[np.invert(np.isnan(df['feature'])), ['feature', 'wt']]
        f = f.loc[np.invert(f['feature'].isin(
            [x for i in feature_meta.loc[feature_meta['Feature_Name'] == feature, 'Special_values'][feature] for x in
             i])), :]
        df.loc[np.isnan(df['feature']), 'feature'] = 'Missing'
        for spl_value in (feature_meta.loc[feature_meta['Feature_Name'] == feature, 'Special_values'][feature]):
            if df.loc[df['feature'].isin(spl_value), :].shape[0] > 0:
                df.loc[df['feature'].isin(spl_value), 'feature'] = str(list(spl_value))

        f['bins'] = pd.cut(f['feature'], feature_boundaries[feature], include_lowest=True, right=False)
        df.loc[f.index, 'feature'] = f['bins']
        df['feature'] = df['feature'].astype('category').cat.set_categories(['Missing'] + [str(x) for x in
                                                                                           feature_meta.loc[
                                                                                               feature_meta[
                                                                                                   'Feature_Name'] == feature, 'Special_values'][
                                                                                               feature]] + f[
                                                                                'bins'].cat.categories.tolist())
        binned_df[feature] = df['feature'].astype('str')

    for feature in embed_features:
        df = pd.DataFrame()
        df['feature'] = features[feature]
        if weight_column is not None:
            #df['labels'] = features[dep_var] * features[weight_column]
            df['wt'] = features[weight_column]
        else:
            #df['labels'] = features[dep_var]
            df['wt'] = 1
        df['feature'] = df['feature'].fillna('Missing')
        for spl_value in (feature_meta.loc[feature_meta['Feature_Name'] == feature, 'Special_values'][feature]):
            if df.loc[df['feature'].isin([str(x) for x in spl_value]), :].shape[0] > 0:
                df.loc[df['feature'].isin([str(x) for x in spl_value]), 'feature'] = str(list(spl_value))
        df['feature'] = df['feature'].astype('category').cat.set_categories(['Missing'] + [str(x) for x in
                                                                                           feature_meta.loc[
                                                                                               feature_meta[
                                                                                                   'Feature_Name'] == feature, 'Special_values'][
                                                                                               feature]] + list(
            set(np.unique(df['feature'])) - set(['Missing'])))
        binned_df[feature] = df['feature'].astype('str')
    
    return binned_df

##### Reading Unified Auto V2 data

In [4]:
vars_= ['subjectrecordtimeoldest', 'subjectrecordtimenewest', 'subjectnewestrecord12month', 'sourcenonderogcount', 'sourcenonderogcount03month', 'sourcenonderogcount06month', 'sourcenonderogcount12month', 'sourcecredheadertimeoldest', 'sourcecredheadertimenewest', 'sourcevoterregistration', 'educationattendance', 'educationevidence', 'educationprogramattended', 'educationinstitutionprivate', 'educationinstitutionrating', 'profliccount', 'proflictypecategory', 'businessassociation', 'businessassociationtimeoldest', 'businesstitleleadership', 'assetownership', 'assetprop', 'assetpropevercount', 'assetpropcurrentcount', 'assetpropcurrenttaxtotal', 'assetproppurchasecount12month', 'assetproppurchasetimeoldest', 'assetproppurchasetimenewest', 'assetpropnewestmortgagetype', 'assetpropeversoldcount', 'assetpropsoldcount12month', 'assetpropsaletimeoldest', 'assetpropsaletimenewest', 'assetpropnewestsaleprice', 'assetpropsalepurchaseratio', 'assetpersonal', 'assetpersonalcount', 'derogcount', 'derogcount12month', 'derogtimenewest', 'criminalfelonycount', 'criminalfelonycount12month', 'criminalfelonytimenewest', 'criminalnonfelonycount', 'criminalnonfelonycount12month', 'criminalnonfelonytimenewest', 'evictioncount', 'evictioncount12month', 'evictiontimenewest', 'lienjudgmentcount', 'lienjudgmentcount12month', 'lienjudgmentsmallclaimscount', 'lienjudgmentcourtcount', 'lienjudgmenttaxcount', 'lienjudgmentforeclosurecount', 'lienjudgmentothercount', 'lienjudgmenttimenewest', 'lienjudgmentdollartotal', 'bankruptcycount', 'bankruptcycount24month', 'bankruptcytimenewest', 'bankruptcychapter', 'bankruptcystatus', 'bankruptcydismissed24month', 'shorttermloanrequest', 'shorttermloanrequest12month', 'shorttermloanrequest24month', 'inquiryauto12month', 'inquirybanking12month', 'inquirytelcom12month', 'inquirynonshortterm12month', 'inquiryshortterm12month', 'inquirycollections12month', 'addronfilecount', 'addronfilecorrectional', 'addronfilecollege', 'addronfilehighrisk', 'addrinputtimeoldest', 'addrinputtimenewest', 'addrinputlengthofres', 'addrinputsubjectcount', 'addrinputsubjectowned', 'addrinputdeedmailing', 'addrinputphoneservice', 'addrinputphonecount', 'addrinputtimelastsale', 'addrinputlastsaleprice', 'addrinputtaxvalue', 'addrinputtaxmarketvalue', 'addrinputavmvalue', 'addrinputavmvalue12month', 'addrinputavmratio12monthprior', 'addrinputavmvalue60month', 'addrinputavmratio60monthprior', 'addrinputcountyratio', 'addrinputtractratio', 'addrinputblockratio', 'addrinputproblems', 'addrcurrenttimeoldest', 'addrcurrenttimenewest', 'addrcurrentlengthofres', 'addrcurrentsubjectowned', 'addrcurrentdeedmailing', 'addrcurrentphoneservice', 'addrcurrenttimelastsale', 'addrcurrentlastsalesprice', 'addrcurrenttaxvalue', 'addrcurrenttaxmarketvalue', 'addrcurrentavmvalue', 'addrcurrentavmvalue12month', 'addrcurrentavmratio12monthprior', 'addrcurrentavmvalue60month', 'addrcurrentavmratio60monthprior', 'addrcurrentcountyratio', 'addrcurrenttractratio', 'addrcurrentblockratio', 'addrcurrentcorrectional', 'addrprevioustimeoldest', 'addrprevioustimenewest', 'addrpreviouslengthofres', 'addrprevioussubjectowned', 'addrpreviouscorrectional', 'addrchangecount03month', 'addrchangecount06month', 'addrchangecount12month', 'addrchangecount24month', 'addrchangecount60month', 'addrlastmovetaxratiodiff', 'addrlastmoveecontrajectory', 'alertregulatorycondition']
len(vars_), len(list(set(vars_)))

(130, 130)

In [5]:
df= pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/1_LN_Auto_All_Updated_Data_220324.csv', 
                usecols= ['unique_id', 'client', 'app_date', 'product', 'fico_pb'] + vars_)
df.shape

(824212, 135)

In [6]:
df['product'].value_counts(dropna= False)

Auto    824212
Name: product, dtype: int64

In [7]:
df.columns= [i.lower() for i in df.columns]

In [8]:
df.head()

,unique_id,client,app_date,fico_pb,product,subjectrecordtimeoldest,subjectrecordtimenewest,subjectnewestrecord12month,sourcenonderogcount,sourcenonderogcount03month,sourcenonderogcount06month,sourcenonderogcount12month,sourcecredheadertimeoldest,sourcecredheadertimenewest,sourcevoterregistration,educationattendance,educationevidence,educationprogramattended,educationinstitutionprivate,educationinstitutionrating,profliccount,proflictypecategory,businessassociation,businessassociationtimeoldest,businesstitleleadership,assetownership,assetprop,assetpropevercount,assetpropcurrentcount,assetpropcurrenttaxtotal,assetproppurchasecount12month,assetproppurchasetimeoldest,assetproppurchasetimenewest,assetpropnewestmortgagetype,assetpropeversoldcount,assetpropsoldcount12month,assetpropsaletimeoldest,assetpropsaletimenewest,assetpropnewestsaleprice,assetpropsalepurchaseratio,assetpersonal,assetpersonalcount,derogcount,derogcount12month,derogtimenewest,criminalfelonycount,criminalfelonycount12month,criminalfelonytimenewest,criminalnonfelonycount,criminalnonfelonycount12month,criminalnonfelonytimenewest,evictioncount,evictioncount12month,evictiontimenewest,lienjudgmentcount,lienjudgmentcount12month,lienjudgmentsmallclaimscount,lienjudgmentcourtcount,lienjudgmenttaxcount,lienjudgmentforeclosurecount,lienjudgmentothercount,lienjudgmenttimenewest,lienjudgmentdollartotal,bankruptcycount,bankruptcycount24month,bankruptcytimenewest,bankruptcychapter,bankruptcystatus,bankruptcydismissed24month,shorttermloanrequest,shorttermloanrequest12month,shorttermloanrequest24month,inquiryauto12month,inquirybanking12month,inquirytelcom12month,inquirynonshortterm12month,inquiryshortterm12month,inquirycollections12month,addronfilecount,addronfilecorrectional,addronfilecollege,addronfilehighrisk,addrinputtimeoldest,addrinputtimenewest,addrinputlengthofres,addrinputsubjectcount,addrinputsubjectowned,addrinputdeedmailing,addrinputphoneservice,addrinputphonecount,addrinputtimelastsale,addrinputlastsaleprice,addrinputtaxvalue,addrinputtaxmarketvalue,addrinputavmvalue,addrinputavmvalue12month,addrinputavmratio12monthprior,addrinputavmvalue60month,addrinputavmratio60monthprior,addrinputcountyratio,addrinputtractratio,addrinputblockratio,addrinputproblems,addrcurrenttimeoldest,addrcurrenttimenewest,addrcurrentlengthofres,addrcurrentsubjectowned,addrcurrentdeedmailing,addrcurrentphoneservice,addrcurrenttimelastsale,addrcurrentlastsalesprice,addrcurrenttaxvalue,addrcurrenttaxmarketvalue,addrcurrentavmvalue,addrcurrentavmvalue12month,addrcurrentavmratio12monthprior,addrcurrentavmvalue60month,addrcurrentavmratio60monthprior,addrcurrentcountyratio,addrcurrenttractratio,addrcurrentblockratio,addrcurrentcorrectional,addrprevioustimeoldest,addrprevioustimenewest,addrpreviouslengthofres,addrprevioussubjectowned,addrpreviouscorrectional,addrchangecount03month,addrchangecount06month,addrchangecount12month,addrchangecount24month,addrchangecount60month,addrlastmovetaxratiodiff,addrlastmoveecontrajectory,alertregulatorycondition
0,453205,SECU,2017-01-01,726.0,Auto,114.0,1.0,1.0,2.0,2.0,2.0,2.0,114.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,-1.0,-1.0,1.0,1.0,1.0,1.0,633700.0,0.0,31.0,31.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,31.0,1.0,30.0,1.0,1.0,0.0,0.0,0.0,31.0,-1.0,633700.0,633700.0,-1.0,-1.0,-1.00,-1.0,-1.00,0.00,0.00,0.00,0.0,31.0,1.0,30.0,1.0,0.0,0.0,31.0,-1.0,633700.0,633700.0,-1.0,-1.0,-1.00,-1.0,-1.00,0.00,0.00,0.00,0.0,16.0,9.0,7.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,12.0,1.0
1,453208,SECU,2017-01-01,624.0,Auto,114.0,1.0,1.0,3.0,2.0,2.0,2.0,114.0,1.0,0.0,1.0,1.0,2.0,0.0,6.0,0.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [9]:
df[vars_].head()

,subjectrecordtimeoldest,subjectrecordtimenewest,subjectnewestrecord12month,sourcenonderogcount,sourcenonderogcount03month,sourcenonderogcount06month,sourcenonderogcount12month,sourcecredheadertimeoldest,sourcecredheadertimenewest,sourcevoterregistration,educationattendance,educationevidence,educationprogramattended,educationinstitutionprivate,educationinstitutionrating,profliccount,proflictypecategory,businessassociation,businessassociationtimeoldest,businesstitleleadership,assetownership,assetprop,assetpropevercount,assetpropcurrentcount,assetpropcurrenttaxtotal,assetproppurchasecount12month,assetproppurchasetimeoldest,assetproppurchasetimenewest,assetpropnewestmortgagetype,assetpropeversoldcount,assetpropsoldcount12month,assetpropsaletimeoldest,assetpropsaletimenewest,assetpropnewestsaleprice,assetpropsalepurchaseratio,assetpersonal,assetpersonalcount,derogcount,derogcount12month,derogtimenewest,criminalfelonycount,criminalfelonycount12month,criminalfelonytimenewest,criminalnonfelonycount,criminalnonfelonycount12month,criminalnonfelonytimenewest,evictioncount,evictioncount12month,evictiontimenewest,lienjudgmentcount,lienjudgmentcount12month,lienjudgmentsmallclaimscount,lienjudgmentcourtcount,lienjudgmenttaxcount,lienjudgmentforeclosurecount,lienjudgmentothercount,lienjudgmenttimenewest,lienjudgmentdollartotal,bankruptcycount,bankruptcycount24month,bankruptcytimenewest,bankruptcychapter,bankruptcystatus,bankruptcydismissed24month,shorttermloanrequest,shorttermloanrequest12month,shorttermloanrequest24month,inquiryauto12month,inquirybanking12month,inquirytelcom12month,inquirynonshortterm12month,inquiryshortterm12month,inquirycollections12month,addronfilecount,addronfilecorrectional,addronfilecollege,addronfilehighrisk,addrinputtimeoldest,addrinputtimenewest,addrinputlengthofres,addrinputsubjectcount,addrinputsubjectowned,addrinputdeedmailing,addrinputphoneservice,addrinputphonecount,addrinputtimelastsale,addrinputlastsaleprice,addrinputtaxvalue,addrinputtaxmarketvalue,addrinputavmvalue,addrinputavmvalue12month,addrinputavmratio12monthprior,addrinputavmvalue60month,addrinputavmratio60monthprior,addrinputcountyratio,addrinputtractratio,addrinputblockratio,addrinputproblems,addrcurrenttimeoldest,addrcurrenttimenewest,addrcurrentlengthofres,addrcurrentsubjectowned,addrcurrentdeedmailing,addrcurrentphoneservice,addrcurrenttimelastsale,addrcurrentlastsalesprice,addrcurrenttaxvalue,addrcurrenttaxmarketvalue,addrcurrentavmvalue,addrcurrentavmvalue12month,addrcurrentavmratio12monthprior,addrcurrentavmvalue60month,addrcurrentavmratio60monthprior,addrcurrentcountyratio,addrcurrenttractratio,addrcurrentblockratio,addrcurrentcorrectional,addrprevioustimeoldest,addrprevioustimenewest,addrpreviouslengthofres,addrprevioussubjectowned,addrpreviouscorrectional,addrchangecount03month,addrchangecount06month,addrchangecount12month,addrchangecount24month,addrchangecount60month,addrlastmovetaxratiodiff,addrlastmoveecontrajectory,alertregulatorycondition
0,114.0,1.0,1.0,2.0,2.0,2.0,2.0,114.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,-1.0,-1.0,1.0,1.0,1.0,1.0,633700.0,0.0,31.0,31.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,31.0,1.0,30.0,1.0,1.0,0.0,0.0,0.0,31.0,-1.0,633700.0,633700.0,-1.0,-1.0,-1.00,-1.0,-1.00,0.00,0.00,0.00,0.0,31.0,1.0,30.0,1.0,0.0,0.0,31.0,-1.0,633700.0,633700.0,-1.0,-1.0,-1.00,-1.0,-1.00,0.00,0.00,0.00,0.0,16.0,9.0,7.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,12.0,1.0
1,114.0,1.0,1.0,3.0,2.0,2.0,2.0,114.0,1.0,0.0,1.0,1.0,2.0,0.0,6.0,0.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,22.0,1.0,21.0,3.0,0.0,1.0,0.0,1.0,-1.0,-1.0,0.0,0.0,192643.0,193097.0,1.00,183511.0,1.05,

In [10]:
df['fico_pb'].isnull().sum()

7299

In [11]:
df['fico_pb'].describe([0.8, 0.85, 0.9, 0.95, 0.99, 0.999, 0.9999])

count     816913.000000
mean         663.750016
std          164.363459
min            0.000000
50%          688.000000
80%          770.000000
85%          788.000000
90%          806.000000
95%          829.000000
99%          865.000000
99.9%        887.000000
99.99%       899.000000
max         9999.000000
Name: fico_pb, dtype: float64

In [12]:
df_lt_660= df[(df['fico_pb']< 660) | (df['fico_pb'].isnull())].reset_index(drop= True)
df_lt_660.shape

(310634, 135)

In [13]:
df_lt_660['fico_pb'].describe(), df_lt_660['fico_pb'].isnull().sum()

(count    303335.000000
 mean        530.915878
 std         197.888859
 min           0.000000
 25%         548.000000
 50%         606.000000
 75%         636.000000
 max         659.000000
 Name: fico_pb, dtype: float64,
 7299)

In [14]:
df_gt_660= df[(df['fico_pb']>= 660)].reset_index(drop= True)
df_gt_660.shape

(513578, 135)

In [15]:
df_gt_660['fico_pb'].describe(), df_gt_660['fico_pb'].isnull().sum()

(count    513578.000000
 mean        742.205953
 std          57.143758
 min         660.000000
 25%         695.000000
 50%         734.000000
 75%         785.000000
 max        9999.000000
 Name: fico_pb, dtype: float64,
 0)

In [16]:
df.shape, df_lt_660.shape, df_gt_660.shape

((824212, 135), (310634, 135), (513578, 135))

#### FICO 660-740

In [17]:
!pip install pypmml


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [18]:
# from pypmml import Model

# auto_ln_lt_660_file_model = Model.load(
#     open('/root/Unified_LN_V2/DataScience-Repo/Unified LN Model/Model PMMLs/unified_LN_lt_660_XG.pmml', 'rb')
#         )
# auto_ln_gt_660_file_model = Model.load(
#     open('/root/Unified_LN_V2/DataScience-Repo/Unified LN Model/Model PMMLs/model_fico_660_740_wt_v2.pmml', 'rb')
#         )

In [19]:
# import sys
# sys.path.append('/root/Unified_LN_V2/DataScience-Repo/Unified LN Model/Binning Codes/')
# from LN_Unified_1_5_Auto_Model_Binning_Code import *

In [20]:
# df_dict= df.to_dict(orient= 'records')
# len(df_dict), df.shape

In [21]:
# %%time

# from tqdm import tqdm

# for d in tqdm(df_dict):
#     binned_d= ln_unified_1_5_auto_binning_function(d, d['fico_pb'])
#     d['p1_v1_5']= ln_model_scoring(binned_d)
#     del binned_d

In [22]:
# del df_gt_660, df_lt_660

In [23]:
import pickle

In [24]:
pwd

'/root/Unified_LN_V2/LN_Auto_V2/Code'

In [25]:
pd.__version__

'1.5.3'

In [26]:
dev_file = df_gt_660.copy()
dep_var = None
weight_column = None
min_bin_size = 0.1
index_col=[]
feature_meta= pd.read_pickle('/root/Model_Score/LN_Model_Files/Auto/FICO_660_740/feature-meta-fico-between-660-740.pickle')
feature_boundaries= pd.read_pickle('/root/Model_Score/LN_Model_Files/Auto/FICO_660_740/feature_boundary_fico_bet_660and740.pickle')
numeric_df= pd.read_pickle('/root/Model_Score/LN_Model_Files/Auto/FICO_660_740/numeric_df_fico_bet_660and740.pickle')
embed_df= pd.read_pickle('/root/Model_Score/LN_Model_Files/Auto/FICO_660_740/embed_df_fico_bet_660and740.pickle')

In [27]:
# Get train bivar tables + get binned train dataframe
binned_df_gt_660 = get_bivar_tables_test(filename=dev_file, predicted=None,
                                                           feature_meta=feature_meta,
                                                           feature_boundaries=feature_boundaries,
                                                           num_features=numeric_df.loc[numeric_df['Use_Feature'] == 'Y', 'Feature_Name'].tolist(),
                                                           embed_features=embed_df.loc[embed_df['Use_Feature'] == 'Y', 'Feature_Name'].tolist(),
                                                           dep_var=dep_var, weight_column=weight_column)

subjectrecordtimeoldest
subjectrecordtimenewest
subjectnewestrecord12month
sourcenonderogcount
sourcenonderogcount03month
sourcenonderogcount06month
sourcenonderogcount12month
sourcecredheadertimeoldest
sourcecredheadertimenewest
sourcevoterregistration
educationattendance
educationevidence
educationprogramattended
educationinstitutionprivate
educationinstitutionrating
profliccount
proflictypecategory
businessassociation
businessassociationtimeoldest
businesstitleleadership
assetownership
assetprop
assetpropevercount
assetpropcurrentcount
assetpropcurrenttaxtotal
assetproppurchasecount12month
assetproppurchasetimeoldest
assetproppurchasetimenewest
assetpropnewestmortgagetype
assetpropeversoldcount
assetpropsoldcount12month
assetpropsaletimeoldest
assetpropsaletimenewest
assetpropnewestsaleprice
assetpropsalepurchaseratio
assetpersonal
assetpersonalcount
derogcount
derogcount12month
derogtimenewest
criminalfelonycount
criminalfelonycount12month
criminalfelonytimenewest
criminalnonfelony

In [28]:
binned_df_gt_660['fico_pb'].describe()

count    513578.000000
mean        742.205953
std          57.143758
min         660.000000
25%         695.000000
50%         734.000000
75%         785.000000
max        9999.000000
Name: fico_pb, dtype: float64

In [29]:
pd.options.display.max_columns= 1000
pd.options.display.max_rows= 1000

In [30]:
binned_df_gt_660[vars_].head()

,subjectrecordtimeoldest,subjectrecordtimenewest,subjectnewestrecord12month,sourcenonderogcount,sourcenonderogcount03month,sourcenonderogcount06month,sourcenonderogcount12month,sourcecredheadertimeoldest,sourcecredheadertimenewest,sourcevoterregistration,educationattendance,educationevidence,educationprogramattended,educationinstitutionprivate,educationinstitutionrating,profliccount,proflictypecategory,businessassociation,businessassociationtimeoldest,businesstitleleadership,assetownership,assetprop,assetpropevercount,assetpropcurrentcount,assetpropcurrenttaxtotal,assetproppurchasecount12month,assetproppurchasetimeoldest,assetproppurchasetimenewest,assetpropnewestmortgagetype,assetpropeversoldcount,assetpropsoldcount12month,assetpropsaletimeoldest,assetpropsaletimenewest,assetpropnewestsaleprice,assetpropsalepurchaseratio,assetpersonal,assetpersonalcount,derogcount,derogcount12month,derogtimenewest,criminalfelonycount,criminalfelonycount12month,criminalfelonytimenewest,criminalnonfelonycount,criminalnonfelonycount12month,criminalnonfelonytimenewest,evictioncount,evictioncount12month,evictiontimenewest,lienjudgmentcount,lienjudgmentcount12month,lienjudgmentsmallclaimscount,lienjudgmentcourtcount,lienjudgmenttaxcount,lienjudgmentforeclosurecount,lienjudgmentothercount,lienjudgmenttimenewest,lienjudgmentdollartotal,bankruptcycount,bankruptcycount24month,bankruptcytimenewest,bankruptcychapter,bankruptcystatus,bankruptcydismissed24month,shorttermloanrequest,shorttermloanrequest12month,shorttermloanrequest24month,inquiryauto12month,inquirybanking12month,inquirytelcom12month,inquirynonshortterm12month,inquiryshortterm12month,inquirycollections12month,addronfilecount,addronfilecorrectional,addronfilecollege,addronfilehighrisk,addrinputtimeoldest,addrinputtimenewest,addrinputlengthofres,addrinputsubjectcount,addrinputsubjectowned,addrinputdeedmailing,addrinputphoneservice,addrinputphonecount,addrinputtimelastsale,addrinputlastsaleprice,addrinputtaxvalue,addrinputtaxmarketvalue,addrinputavmvalue,addrinputavmvalue12month,addrinputavmratio12monthprior,addrinputavmvalue60month,addrinputavmratio60monthprior,addrinputcountyratio,addrinputtractratio,addrinputblockratio,addrinputproblems,addrcurrenttimeoldest,addrcurrenttimenewest,addrcurrentlengthofres,addrcurrentsubjectowned,addrcurrentdeedmailing,addrcurrentphoneservice,addrcurrenttimelastsale,addrcurrentlastsalesprice,addrcurrenttaxvalue,addrcurrenttaxmarketvalue,addrcurrentavmvalue,addrcurrentavmvalue12month,addrcurrentavmratio12monthprior,addrcurrentavmvalue60month,addrcurrentavmratio60monthprior,addrcurrentcountyratio,addrcurrenttractratio,addrcurrentblockratio,addrcurrentcorrectional,addrprevioustimeoldest,addrprevioustimenewest,addrpreviouslengthofres,addrprevioussubjectowned,addrpreviouscorrectional,addrchangecount03month,addrchangecount06month,addrchangecount12month,addrchangecount24month,addrchangecount60month,addrlastmovetaxratiodiff,addrlastmoveecontrajectory,alertregulatorycondition
0,"[73.5, 234.5)","[-inf, 1.43)","[0.991, inf)","[1.5, 2.5)","[1.5, inf)","[1.5, inf)","[1.5, inf)","[73.5, 292.5)","[-inf, 1.893)","[-inf, 0.5)","[-inf, 0.0892)","[-inf, 0.5)","[-inf, 0.162)",[-1],"[-inf, 0.265)","[-inf, 0.0435)",[-1],"[-inf, 0.5)",[-1],[-1],"[0.5, inf)","[0.5, inf)","[0.5, inf)","[0.5, inf)","[122517.5, inf)","[-inf, 0.0781)","[-inf, 130.282)","[-inf, 90.017)","[-inf, 0.0809)","[-inf, 0.5)","[-inf, 0.0237)",[-1],[-1],[-1],"[-inf, 0.133)","[-inf, 0.0199)","[-inf, 0.0312)","[-inf, 0.5)","[-inf, 0.0108)",[-1],"[-inf, 0.00698)","[-inf, 0.000239)",[-1],"[-inf, 0.0331)","[-inf, 0.00176)",[-1],"[-inf, 0.00292)","[-inf, 0.000441)",[-1],"[-inf, 0.0988)","[-inf, 0.00751)","[-inf, 0.00957)","[-inf, 0.0371)","[-inf, 0.0496)","[0.0, inf)","[-inf, 0.00241)",[-1],"[-inf, 994.437)","[-inf, 0.0493)","[-inf, 0.00463)",[-1],"[-inf, 0.0522)","[-inf, 0.0491)","[-inf, 0.000147)","[-inf, 0.00277)","[-inf, 0.000294)","[-inf, 0.000588)","[-inf, 0.5)","[-inf, 0.5)","[-inf, 0.0304)","[-inf, 0.5)","[-inf, 0.0

In [31]:
binned_df_gt_660.shape

(513578, 135)

In [32]:
binned_df_gt_660[vars_].isnull().sum()

subjectrecordtimeoldest            0
subjectrecordtimenewest            0
subjectnewestrecord12month         0
sourcenonderogcount                0
sourcenonderogcount03month         0
sourcenonderogcount06month         0
sourcenonderogcount12month         0
sourcecredheadertimeoldest         0
sourcecredheadertimenewest         0
sourcevoterregistration            0
educationattendance                0
educationevidence                  0
educationprogramattended           0
educationinstitutionprivate        0
educationinstitutionrating         0
profliccount                       0
proflictypecategory                0
businessassociation                0
businessassociationtimeoldest      0
businesstitleleadership            0
assetownership                     0
assetprop                          0
assetpropevercount                 0
assetpropcurrentcount              0
assetpropcurrenttaxtotal           0
assetproppurchasecount12month      0
assetproppurchasetimeoldest        0
a

In [35]:
del dev_file
del dep_var
del weight_column
del min_bin_size
# del config_file
del index_col
del feature_meta
del feature_boundaries
del numeric_df
del embed_df

#### FICO < 660

In [37]:
path = '/root/Model_Score/LN_Model_Files/Auto/FICO_lt_660/'

dev_file = df_lt_660 #'MCMC_data_score/MCMC_Unified_LN_Auto_120124/MCMC_lt_660_120124.csv'
feature_meta = pd.read_pickle(path + 'feature_meta.pickle')

feature_boundaries = pd.read_pickle(path + 'feature_boundaries.pickle')

numeric_df = pd.read_pickle(path + 'numeric_df.pickle')

embed_df = pd.read_pickle(path + 'embed_df.pickle')
dep_var = None
weight_column = None


In [38]:
binned_df_lt_660 = get_bivar_tables_test(filename=dev_file, predicted=None,
                                                                                   feature_meta=feature_meta,
                                                                                   feature_boundaries=feature_boundaries,
                                                                                   num_features=numeric_df.loc[
                                                                                       numeric_df[
                                                                                           'Use_Feature'] == 'Y', 'Feature_Name'].tolist(),
                                                                                   embed_features=embed_df.loc[embed_df[
                                                                                                                   'Use_Feature'] == 'Y', 'Feature_Name'].tolist(),
                                                                                   dep_var=dep_var,
                                                                                   weight_column=weight_column)

subjectrecordtimeoldest
subjectrecordtimenewest
subjectnewestrecord12month
sourcenonderogcount
sourcenonderogcount03month
sourcenonderogcount06month
sourcenonderogcount12month
sourcecredheadertimeoldest
sourcecredheadertimenewest
sourcevoterregistration
educationattendance
educationevidence
educationprogramattended
educationinstitutionprivate
educationinstitutionrating
profliccount
proflictypecategory
businessassociation
businessassociationtimeoldest
businesstitleleadership
assetownership
assetprop
assetpropevercount
assetpropcurrentcount
assetpropcurrenttaxtotal
assetproppurchasecount12month
assetproppurchasetimeoldest
assetproppurchasetimenewest
assetpropnewestmortgagetype
assetpropeversoldcount
assetpropsoldcount12month
assetpropsaletimeoldest
assetpropsaletimenewest
assetpropnewestsaleprice
assetpropsalepurchaseratio
assetpersonal
assetpersonalcount
derogcount
derogcount12month
derogtimenewest
criminalfelonycount
criminalfelonycount12month
criminalfelonytimenewest
criminalnonfelony

In [39]:
binned_df_lt_660.head()

,unique_id,client,app_date,fico_pb,product,subjectrecordtimeoldest,subjectrecordtimenewest,subjectnewestrecord12month,sourcenonderogcount,sourcenonderogcount03month,sourcenonderogcount06month,sourcenonderogcount12month,sourcecredheadertimeoldest,sourcecredheadertimenewest,sourcevoterregistration,educationattendance,educationevidence,educationprogramattended,educationinstitutionprivate,educationinstitutionrating,profliccount,proflictypecategory,businessassociation,businessassociationtimeoldest,businesstitleleadership,assetownership,assetprop,assetpropevercount,assetpropcurrentcount,assetpropcurrenttaxtotal,assetproppurchasecount12month,assetproppurchasetimeoldest,assetproppurchasetimenewest,assetpropnewestmortgagetype,assetpropeversoldcount,assetpropsoldcount12month,assetpropsaletimeoldest,assetpropsaletimenewest,assetpropnewestsaleprice,assetpropsalepurchaseratio,assetpersonal,assetpersonalcount,derogcount,derogcount12month,derogtimenewest,criminalfelonycount,criminalfelonycount12month,criminalfelonytimenewest,criminalnonfelonycount,criminalnonfelonycount12month,criminalnonfelonytimenewest,evictioncount,evictioncount12month,evictiontimenewest,lienjudgmentcount,lienjudgmentcount12month,lienjudgmentsmallclaimscount,lienjudgmentcourtcount,lienjudgmenttaxcount,lienjudgmentforeclosurecount,lienjudgmentothercount,lienjudgmenttimenewest,lienjudgmentdollartotal,bankruptcycount,bankruptcycount24month,bankruptcytimenewest,bankruptcychapter,bankruptcystatus,bankruptcydismissed24month,shorttermloanrequest,shorttermloanrequest12month,shorttermloanrequest24month,inquiryauto12month,inquirybanking12month,inquirytelcom12month,inquirynonshortterm12month,inquiryshortterm12month,inquirycollections12month,addronfilecount,addronfilecorrectional,addronfilecollege,addronfilehighrisk,addrinputtimeoldest,addrinputtimenewest,addrinputlengthofres,addrinputsubjectcount,addrinputsubjectowned,addrinputdeedmailing,addrinputphoneservice,addrinputphonecount,addrinputtimelastsale,addrinputlastsaleprice,addrinputtaxvalue,addrinputtaxmarketvalue,addrinputavmvalue,addrinputavmvalue12month,addrinputavmratio12monthprior,addrinputavmvalue60month,addrinputavmratio60monthprior,addrinputcountyratio,addrinputtractratio,addrinputblockratio,addrinputproblems,addrcurrenttimeoldest,addrcurrenttimenewest,addrcurrentlengthofres,addrcurrentsubjectowned,addrcurrentdeedmailing,addrcurrentphoneservice,addrcurrenttimelastsale,addrcurrentlastsalesprice,addrcurrenttaxvalue,addrcurrenttaxmarketvalue,addrcurrentavmvalue,addrcurrentavmvalue12month,addrcurrentavmratio12monthprior,addrcurrentavmvalue60month,addrcurrentavmratio60monthprior,addrcurrentcountyratio,addrcurrenttractratio,addrcurrentblockratio,addrcurrentcorrectional,addrprevioustimeoldest,addrprevioustimenewest,addrpreviouslengthofres,addrprevioussubjectowned,addrpreviouscorrectional,addrchangecount03month,addrchangecount06month,addrchangecount12month,addrchangecount24month,addrchangecount60month,addrlastmovetaxratiodiff,addrlastmoveecontrajectory,alertregulatorycondition
0,453208,SECU,2017-01-01,624.0,Auto,"[58.5, 141.5)","[-inf, 1.608)","[0.981, inf)","[2.5, 3.5)","[1.5, inf)","[1.5, inf)","[1.5, inf)","[58.5, 141.5)","[-inf, 2.019)","[-inf, 0.5)","[0.0736, inf)","[0.0981, inf)","[0.127, inf)","[-inf, 0.0928)","[0.23, inf)","[-inf, 0.035)",[-1],"[-inf, 0.0929)",[-1],[-1],"[-inf, 0.5)","[-inf, 0.5)","[-inf, 0.5)","[-inf, 0.5)",[-1],"[-inf, 0.0423)",[-1],[-1],[-1],"[-inf, 0.5)","[-inf, 0.0121)",[-1],[-1],[-1],"[-inf, 0.0764)","[-inf, 0.0199)","[-inf, 0.0324)","[-inf, 0.358)","[-inf, 0.0378)",[-1],"[-inf, 0.0137)","[-inf, 0.000461)",[-1],"[-inf, 0.0496)","[-inf, 0.00272)",[-1],"[-inf, 0.013)","[-inf, 0.00154)",[-1],"[-inf, 0.206)","[-inf, 0.0254)","[-inf, 0.0253)","[-inf, 0.0943)","[-inf, 0.0842)","[0.0, inf)","[-inf, 0.00257)",[-1],"[-inf, 839.0)","[-inf, 0.0753)","[-inf, 0.0211)",[-1],"[-inf, 0.0796)","[-inf, 0.0749)","[-inf, 0.000461)","[-inf, 0.00484)","[-inf, 0.000307)","[-inf, 0.000768)","[-inf, 0.5)","[-inf, 0.5)","[-inf

In [40]:
binned_df_lt_660.shape

(310634, 135)

In [41]:
binned_df_lt_660['fico_pb'].describe()

count    303335.000000
mean        530.915878
std         197.888859
min           0.000000
25%         548.000000
50%         606.000000
75%         636.000000
max         659.000000
Name: fico_pb, dtype: float64

In [42]:
binned_df_lt_660[vars_].isnull().sum().sort_values(ascending= False)

alertregulatorycondition           0
subjectrecordtimeoldest            0
addrcurrenttractratio              0
addrcurrentblockratio              0
addrcurrentcorrectional            0
addrprevioustimeoldest             0
addrprevioustimenewest             0
addrpreviouslengthofres            0
addrprevioussubjectowned           0
addrpreviouscorrectional           0
educationattendance                0
educationevidence                  0
educationprogramattended           0
educationinstitutionprivate        0
educationinstitutionrating         0
profliccount                       0
proflictypecategory                0
businessassociation                0
businessassociationtimeoldest      0
businesstitleleadership            0
assetownership                     0
assetprop                          0
assetpropevercount                 0
assetpropcurrentcount              0
assetpropcurrenttaxtotal           0
assetproppurchasecount12month      0
assetproppurchasetimeoldest        0
a

#### Scoring through Models

In [43]:
model_lt_660= h2o.import_mojo('/root/Model_Score/LN_Model_Files/Auto/FICO_lt_660/unified_LN_lt_660_XG.zip')
model_lt_660

generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGenericEstimator : Import MOJO Model
Model Key: Generic_model_python_1712212377309_1


Model Summary: 
    number_of_trees
--  -----------------
    700

ModelMetricsBinomialGeneric: generic
** Reported on train data. **

MSE: 0.061814763781912335
RMSE: 0.248625750440119
LogLoss: 0.23801108770082613
Mean Per-Class Error: 0.401014774862162
AUC: 0.6559332590187593
AUCPR: 0.11806542169483741
Gini: 0.31186651803751864

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.08839001069672851
       0      1      Error    Rate
-----  -----  -----  -------  -----------------
0      63072  17081  0.2131   (17081.0/80153.0)
1      3426   2391   0.589    (3426.0/5817.0)
Total  66498  19472  0.2385   (20507.0/85970.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.08839      0.189129  169
max f2                       0.0564947    0.312435  270
max f0point5                 0.135875     0.161961  60
max accuracy                 0.188945     0.932125  0
max precision                0.15857      0.209827  21
max recall                   0.0206055    1         398
max specificity              0.188945     0.999784  0
max absolute_mcc             0.0791681    0.122218  195
max min_per_class_accuracy   0.066646     0.610107  235
max mean_per_class_accuracy  0.0636533    0.615662  245
max tns                      0.188945     80136.2   0
max fns                      0.188945     5817.93   0
max fps                      0.0202076    80153.6   399
max tps                      0.0206055    5817.93   398
max tnr                      0.188945     0.999784  0
max fnr                      0.188945     1         0
max fpr                      0.0202076    1         399
max tpr                      0.0206055    1         398

Gains/Lift Table: Avg response rate:  6.86 %, avg score:  7.88 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain        cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  ----------  -----------------  --------------------
1        0.00562813                  0.165366           2.48303   2.48303            0.170316         0.192682   0.170316                    0.192682            0.0139748       0.0139748                  148.303     148.303            0.00896139
2        0.0164188                   0.151918           2.68268   2.61424            0.18401          0.173663   0.179316                    0.180183            0.0289479       0.0429227                  168.268     161.424            0.0284558
3        0.027278                    0.143333           2.94153   2.74453            0.201765         0.164537   0.188253                    0.173954            0.0319425       0.0748652                  194.153     174.453            0.0510918
4        0.0378906                   0.138996           2.10689   2.56594            0.144516         0.158103   0.176003                    0.169514            0.0223598       0.097225                   110.689     156.594            0.063704
5        0.0487908                   0.134872           2.27109   2.50007            0.155779         0.154103   0.171485                    0.166071            0.0247554       0.12198                    127.109     150.007            0.0785795
6        0.100868                    0.118799           1.71743   2.096              0.117802         0.143407   0.143769                    0.15437             0.089439        0.211419                   71.7427     109.6              0.118693
7        0.152507                 

In [44]:
model_gt_660= h2o.import_mojo('/root/Model_Score/LN_Model_Files/Auto/FICO_660_740/model_fico_660_740_wt_v2.zip')
model_gt_660

generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGenericEstimator : Import MOJO Model
Model Key: Generic_model_python_1712212377309_2


Model Summary: 
    number_of_trees
--  -----------------
    1800

ModelMetricsBinomialGeneric: generic
** Reported on train data. **

MSE: 0.02520647850051801
RMSE: 0.15876548271119265
LogLoss: 0.11526250944157881
Mean Per-Class Error: 0.4349961051765714
AUC: 0.6941331959416909
AUCPR: 0.06302346957471204
Gini: 0.38826639188338175

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.06259425453146598
       0      1     Error    Rate
-----  -----  ----  -------  ----------------
0      82759  3844  0.0444   (3844.0/86603.0)
1      1930   407   0.8258   (1930.0/2337.0)
Total  84689  4251  0.0649   (5774.0/88940.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.0625943    0.12376   97
max f2                       0.0380269    0.189002  182
max f0point5                 0.0750356    0.115228  66
max accuracy                 0.170462     0.973681  0
max precision                0.13283      0.304134  7
max recall                   0.00800817   1         398
max specificity              0.170462     0.999974  0
max absolute_mcc             0.0490585    0.101694  140
max min_per_class_accuracy   0.0273652    0.630888  240
max mean_per_class_accuracy  0.0228127    0.636587  273
max tns                      0.170462     86601.4   0
max fns                      0.170462     2338.56   0
max fps                      0.00762083   86603.7   399
max tps                      0.00800817   2338.56   398
max tnr                      0.170462     0.999974  0
max fnr                      0.170462     1         0
max fpr                      0.00762083   1         399
max tpr                      0.00800817   1         398

Gains/Lift Table: Avg response rate:  2.63 %, avg score:  4.03 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0134996                   0.0832442          4.4001    4.4001             0.115776         0.112145   0.115776                    0.112145            0.0593995       0.0593995                  340.01    340.01             0.0471403
2        0.0234268                   0.0777937          4.27394   4.34664            0.112457         0.0957804  0.11437                     0.105211            0.0424282       0.101828                   327.394   334.664            0.0805196
3        0.0353119                   0.0709982          3.73462   4.14065            0.0982659        0.092973   0.108949                    0.101092            0.0443864       0.146214                   273.462   314.065            0.113899
4        0.0472829                   0.0653245          3.32614   3.93443            0.0875179        0.0891957  0.103523                    0.0980799           0.0398172       0.186031                   232.614   293.443            0.142498
5        0.0576566                   0.0622425          2.45398   3.66806            0.0645695        0.0909368  0.0965147                   0.0967947           0.0254569       0.211488                   145.398   266.806            0.157989
6        0.10913                     0.0471005          1.99093   2.87701            0.0523857        0.082614   0.0757003                   0.0901061           0.10248         0.313969                   99.0931   187.701            0.210374
7        0.155469                    0.0391887          1.52

In [45]:
df_lt_660_h2o= h2o.H2OFrame(binned_df_lt_660)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [46]:
df_gt_660_h2o= h2o.H2OFrame(binned_df_gt_660)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [47]:
df_lt_660_h2o.shape, df_gt_660_h2o.shape, df_lt_660.shape, df_gt_660.shape, binned_df_lt_660.shape, binned_df_gt_660.shape

((310634, 135),
 (513578, 135),
 (310634, 135),
 (513578, 135),
 (310634, 135),
 (513578, 135))

In [48]:
binned_df_lt_660['p1_v1_5']= model_lt_660.predict(df_lt_660_h2o).as_data_frame()['p1']
binned_df_lt_660['p1_v1_5'].describe()

generic prediction progress: |███████████████████████████████████████████████████| (done) 100%


/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


count    310634.000000
mean          0.067326
std           0.033087
min           0.019896
25%           0.042442
50%           0.059018
75%           0.087370
max           0.190158
Name: p1_v1_5, dtype: float64

In [49]:
binned_df_gt_660['p1_v1_5']= model_gt_660.predict(df_gt_660_h2o).as_data_frame()['p1']
binned_df_gt_660['p1_v1_5'].describe()

generic prediction progress: |███████████████████████████████████████████████████| (done) 100%


/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


count    513578.000000
mean          0.020520
std           0.013315
min           0.007456
25%           0.011802
50%           0.016178
75%           0.024474
max           0.176066
Name: p1_v1_5, dtype: float64

In [50]:
binned_df_gt_660.head(2)

,unique_id,client,app_date,fico_pb,product,subjectrecordtimeoldest,subjectrecordtimenewest,subjectnewestrecord12month,sourcenonderogcount,sourcenonderogcount03month,sourcenonderogcount06month,sourcenonderogcount12month,sourcecredheadertimeoldest,sourcecredheadertimenewest,sourcevoterregistration,educationattendance,educationevidence,educationprogramattended,educationinstitutionprivate,educationinstitutionrating,profliccount,proflictypecategory,businessassociation,businessassociationtimeoldest,businesstitleleadership,assetownership,assetprop,assetpropevercount,assetpropcurrentcount,assetpropcurrenttaxtotal,assetproppurchasecount12month,assetproppurchasetimeoldest,assetproppurchasetimenewest,assetpropnewestmortgagetype,assetpropeversoldcount,assetpropsoldcount12month,assetpropsaletimeoldest,assetpropsaletimenewest,assetpropnewestsaleprice,assetpropsalepurchaseratio,assetpersonal,assetpersonalcount,derogcount,derogcount12month,derogtimenewest,criminalfelonycount,criminalfelonycount12month,criminalfelonytimenewest,criminalnonfelonycount,criminalnonfelonycount12month,criminalnonfelonytimenewest,evictioncount,evictioncount12month,evictiontimenewest,lienjudgmentcount,lienjudgmentcount12month,lienjudgmentsmallclaimscount,lienjudgmentcourtcount,lienjudgmenttaxcount,lienjudgmentforeclosurecount,lienjudgmentothercount,lienjudgmenttimenewest,lienjudgmentdollartotal,bankruptcycount,bankruptcycount24month,bankruptcytimenewest,bankruptcychapter,bankruptcystatus,bankruptcydismissed24month,shorttermloanrequest,shorttermloanrequest12month,shorttermloanrequest24month,inquiryauto12month,inquirybanking12month,inquirytelcom12month,inquirynonshortterm12month,inquiryshortterm12month,inquirycollections12month,addronfilecount,addronfilecorrectional,addronfilecollege,addronfilehighrisk,addrinputtimeoldest,addrinputtimenewest,addrinputlengthofres,addrinputsubjectcount,addrinputsubjectowned,addrinputdeedmailing,addrinputphoneservice,addrinputphonecount,addrinputtimelastsale,addrinputlastsaleprice,addrinputtaxvalue,addrinputtaxmarketvalue,addrinputavmvalue,addrinputavmvalue12month,addrinputavmratio12monthprior,addrinputavmvalue60month,addrinputavmratio60monthprior,addrinputcountyratio,addrinputtractratio,addrinputblockratio,addrinputproblems,addrcurrenttimeoldest,addrcurrenttimenewest,addrcurrentlengthofres,addrcurrentsubjectowned,addrcurrentdeedmailing,addrcurrentphoneservice,addrcurrenttimelastsale,addrcurrentlastsalesprice,addrcurrenttaxvalue,addrcurrenttaxmarketvalue,addrcurrentavmvalue,addrcurrentavmvalue12month,addrcurrentavmratio12monthprior,addrcurrentavmvalue60month,addrcurrentavmratio60monthprior,addrcurrentcountyratio,addrcurrenttractratio,addrcurrentblockratio,addrcurrentcorrectional,addrprevioustimeoldest,addrprevioustimenewest,addrpreviouslengthofres,addrprevioussubjectowned,addrpreviouscorrectional,addrchangecount03month,addrchangecount06month,addrchangecount12month,addrchangecount24month,addrchangecount60month,addrlastmovetaxratiodiff,addrlastmoveecontrajectory,alertregulatorycondition,p1_v1_5
0,453205,SECU,2017-01-01,726.0,Auto,"[73.5, 234.5)","[-inf, 1.43)","[0.991, inf)","[1.5, 2.5)","[1.5, inf)","[1.5, inf)","[1.5, inf)","[73.5, 292.5)","[-inf, 1.893)","[-inf, 0.5)","[-inf, 0.0892)","[-inf, 0.5)","[-inf, 0.162)",[-1],"[-inf, 0.265)","[-inf, 0.0435)",[-1],"[-inf, 0.5)",[-1],[-1],"[0.5, inf)","[0.5, inf)","[0.5, inf)","[0.5, inf)","[122517.5, inf)","[-inf, 0.0781)","[-inf, 130.282)","[-inf, 90.017)","[-inf, 0.0809)","[-inf, 0.5)","[-inf, 0.0237)",[-1],[-1],[-1],"[-inf, 0.133)","[-inf, 0.0199)","[-inf, 0.0312)","[-inf, 0.5)","[-inf, 0.0108)",[-1],"[-inf, 0.00698)","[-inf, 0.000239)",[-1],"[-inf, 0.0331)","[-inf, 0.00176)",[-1],"[-inf, 0.00292)","[-inf, 0.000441)",[-1],"[-inf, 0.0988)","[-inf, 0.00751)","[-inf, 0.00957)","[-inf, 0.0371)","[-inf, 0.0496)","[0.0, inf)","[-inf, 0.00241)",[-1],"[-inf, 994.437)","[-inf, 0.0493)","[-inf, 0.00463)",[-1],"[-inf, 0.0522)","[-inf, 0.0491)","[-inf, 0.000147)","[-inf, 0.00277)","[-inf, 0.000294)","[-in

In [51]:
binned_df_gt_660['p1_v1_5'].isnull().sum(), binned_df_lt_660['p1_v1_5'].isnull().sum()

(0, 0)

In [52]:
binned_df_lt_660['dataset']= 'fico_lt_660'
binned_df_gt_660['dataset']= 'fico_gt_660'

In [58]:
df_scored= pd.concat([binned_df_lt_660[['unique_id', 'client', 'p1_v1_5', 'fico_pb', 'dataset']],
                         binned_df_gt_660[['unique_id', 'client', 'p1_v1_5', 'fico_pb', 'dataset']]]).reset_index(drop= True)
df_scored.head()

,unique_id,client,p1_v1_5,fico_pb,dataset
0,453208,SECU,0.053346,624.0,fico_lt_660
1,453394,SECU,0.064319,0.0,fico_lt_660
2,453515,SECU,0.086988,614.0,fico_lt_660
3,453555,SECU,0.035332,637.0,fico_lt_660
4,453594,SECU,0.094630,587.0,fico_lt_660


In [59]:
df.shape, df_scored.shape

((824212, 135), (824212, 5))

In [60]:
df_scored['dataset'].value_counts(dropna= False)

fico_gt_660    513578
fico_lt_660    310634
Name: dataset, dtype: int64

In [61]:
df_scored.groupby('dataset')['fico_pb'].agg(['min', 'max'])

,min,max
dataset,,
fico_gt_660,660.0,9999.0
fico_lt_660,0.0,659.0


In [62]:
binned_df_lt_660.shape[0] + binned_df_gt_660.shape[0]

824212

In [63]:
df_lt_660.shape[0] + df_gt_660.shape[0]

824212

In [64]:
df_scored.isnull().sum()

unique_id       0
client          0
p1_v1_5         0
fico_pb      7299
dataset         0
dtype: int64

In [65]:
df_scored.describe()

,p1_v1_5,fico_pb
count,824212.000000,816913.000000
mean,0.038160,663.750016
std,0.032211,164.363459
min,0.007456,0.000000
25%,0.014175,628.000000
50%,0.025724,688.000000
75%,0.052233,753.000000
max,0.190158,9999.000000


lt_vars= """inquiryauto12month
inquirynonshortterm12month
assetpropevercount
addrlastmovetaxratiodiff
assetpropcurrentcount
addronfilecount
addrlastmoveecontrajectory
inquirybanking12month
sourcenonderogcount
addrinputblockratio
addrcurrentsubjectowned
addrchangecount60month
assetpropnewestmortgagetype
addrinputcountyratio
addrchangecount12month
addrchangecount24month
addrinputtractratio
addrinputdeedmailing
addrinputsubjectowned
addrinputavmratio60monthprior
addrinputsubjectcount
assetproppurchasetimeoldest
derogtimenewest
sourcevoterregistration
sourcenonderogcount06month
sourcenonderogcount03month
addrinputavmratio12monthprior""".splitlines()
lt_vars

for var in lt_vars:
    print(var, binned_df_lt_660[var].sort_values().unique())

In [66]:
# df_scored.to_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/040424_Auto_LN_V1_5_Scored_Data.csv', index= False)